## Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Download dataset

In [ ]:
!wget https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip -O opus.zip
!unzip opus.zip

with open("/content/TEP.en-fa.en", 'r', encoding='utf-8') as f:
    input_sentences = f.read().split('\n')
f.close()

with open("/content/TEP.en-fa.fa", 'r', encoding='utf-8') as f:
    output_sentences = f.read().split('\n')
f.close()

import csv

header = ['english_sentence', 'persian_sentence']

with open('corpus.csv', 'w', encoding='utf-8', newline='') as corpus_writer:
    writer = csv.writer(corpus_writer)

    # write the header
    writer.writerow(header)

    for value in range(len(output_sentences)):
        writer.writerow([input_sentences[value], output_sentences[value]])

corpus_writer.close()

--2022-07-17 15:59:06--  https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip [following]
--2022-07-17 15:59:08--  https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16353318 (16M) [application/zip]
Saving to: ‘opus.zip’

opus.zip            100%[===================>]  15.60M  9.46MB/s    in 1.6s    

2022-07-17 15:59:10 (9.46 MB/s) - ‘opus.zip’ saved [16353318/16353318]

Archive:  opus.zip
  inflating: TEP.en-fa.en            
  inflating: TEP.en-fa.fa            
  inflating: README                  


## Make our data ready

### Read CSV file

In [ ]:
lines=pd.read_csv("/content/corpus.csv",encoding='utf-8')

In [ ]:
lines

,english_sentence,persian_sentence
0,raspy breathing .,صداي خر خر .
1,dad .,پدر .
2,maybe its the wind .,شايد صداي باد باشه .
3,no .,نه .
4,stop please stop .,دست نگه داريد خواهش ميکنم دست نگه داريد .
...,...,...
612082,zodiacal light .,حمره مغربيه .
612083,zombi .,انسان زنده شد .
612084,zombiism .,مارخداگرائي .
612085,zonal .,مداري .


In [ ]:
lines.head(20)

,english_sentence,persian_sentence
0,raspy breathing .,صداي خر خر .
1,dad .,پدر .
2,maybe its the wind .,شايد صداي باد باشه .
3,no .,نه .
4,stop please stop .,دست نگه داريد خواهش ميکنم دست نگه داريد .
5,"you have a week , evans then well burn the house .",اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواهيم سوزوند .
6,william .,ويليام .
7,"god damn it , william .",لعنتي . ويليام 8 .
8,god damn it put that down .,لعنت به تو . اونو بذار زمين .
9,let go .,بذار برم .
